## 1. Создание базы и таблиц

Подключимся к базе

In [60]:
import psycopg2

con = psycopg2.connect(
  database="russian-news", 
  user="postgres", 
  password="passcode4854", 
  host="127.0.0.1", 
  port="5432"
)

print("Database opened successfully")

Database opened successfully


Создадим таблицу, зададим ее структуру. Нам нужны 5 столбцов: id, издание, дата публикации, название, текст

In [61]:
con = psycopg2.connect(
  database="russian-news", 
  user="postgres", 
  password="passcode4854", 
  host="127.0.0.1", 
  port="5432"
)
cur = con.cursor()  
cur.execute('''CREATE TABLE corpus  
     (id SERIAL,
     newspaper TEXT NOT NULL,
     date TEXT NOT NULL,
     title TEXT NOT NULL,
     article TEXT NOT NULL);''')

print("Table created successfully")
con.commit()  
con.close()

Table created successfully


Возьмем по 100 статей Ленты, РИА-новостей и Известий за 2001, 2002 и 2003 год. Поместим их в список кортежей и последовательно запишем в таблицу.

In [62]:
import os
import unicodedata

In [63]:
%%time
corp = []
for i in range(2001, 2004):
    for fl in os.listdir('data/lenta/lenta{}/'.format(i))[:100]:
        with open('data/lenta/lenta{}/'.format(i) + fl, 'r', encoding='UTF-8') as f:
            file = f.readlines()
            tuple_line = ('lenta', unicodedata.normalize("NFKD",file[0]).strip('\n'), unicodedata.normalize("NFKD",file[1]).strip('\n'), ' '.join(unicodedata.normalize("NFKD",file) for file in file[2:]))
            corp.append(tuple_line)

Wall time: 209 ms


In [64]:
%%time
for i in range(2001, 2004):
    for fl in os.listdir('data/izvestiya/izvestiya{}/'.format(i))[:100]:
        with open('data/izvestiya/izvestiya{}/'.format(i) + fl, 'r', encoding='UTF-8') as f:
            file = f.readlines()
            tuple_line = ('izvestiya', unicodedata.normalize("NFKD",file[0]).strip('\n'), unicodedata.normalize("NFKD",file[1]).strip('\n'), ' '.join(unicodedata.normalize("NFKD",file) for file in file[2:]))
            corp.append(tuple_line)

Wall time: 113 ms


In [65]:
%%time
for i in range(2001, 2004):
    for fl in os.listdir('data/ria/ria{}/'.format(i))[:100]:
        with open('data/ria/ria{}/'.format(i) + fl, 'r', encoding='UTF-8') as f:
            file = f.readlines()
            tuple_line = ('ria', unicodedata.normalize("NFKD",file[0]).strip('\n'), unicodedata.normalize("NFKD",file[1]).strip('\n'), ' '.join(unicodedata.normalize("NFKD",file) for file in file[2:]))
            corp.append(tuple_line)

Wall time: 440 ms


In [67]:
%%time
con = psycopg2.connect(
  database="russian-news", 
  user="postgres", 
  password="passcode4854", 
  host="127.0.0.1", 
  port="5432"
)
user_records = ", ".join(["%s"] * len(corp))

insert_query = (
    f"INSERT INTO corpus (newspaper, date, title, article) VALUES {user_records}"
)

cursor = con.cursor()
cursor.execute(insert_query, corp)
con.commit()  
con.close()

Wall time: 846 ms


Создадим еще одну таблицу, куда запишем количество слов в статьях. Нам понадобятся три столбца: id, название и количество слов:

In [68]:
%%time
n_words = []
for i in corp:
    tuple_line = (i[2], len(i[3].split()))
    n_words.append(tuple_line)

Wall time: 34.5 ms


In [69]:
con = psycopg2.connect(
  database="russian-news", 
  user="postgres", 
  password="passcode4854", 
  host="127.0.0.1", 
  port="5432"
)
cur = con.cursor()  
cur.execute('''CREATE TABLE corpus_words  
     (id SERIAL,
     title TEXT NOT NULL,
     n_words INT NOT NULL);''')

print("Table created successfully")
con.commit()  
con.close()

Table created successfully


In [70]:
%%time
con = psycopg2.connect(
  database="russian-news", 
  user="postgres", 
  password="passcode4854", 
  host="127.0.0.1", 
  port="5432"
)
user_records = ", ".join(["%s"] * len(n_words))

insert_query = (
    f"INSERT INTO corpus_words (title, n_words) VALUES {user_records}"
)

cursor = con.cursor()
cursor.execute(insert_query, n_words)
con.commit()  
con.close()

Wall time: 103 ms


Создадим третью таблицу, куда запишем количество предложений в статьях. Понадобятся 3 столбца: id, название и количество предложений:

In [71]:
n_sents = []
for i in corp:
    tuple_line = (i[2], len(i[3].split('.')))
    n_sents.append(tuple_line)

In [72]:
con = psycopg2.connect(
  database="russian-news", 
  user="postgres", 
  password="passcode4854", 
  host="127.0.0.1", 
  port="5432"
)
cur = con.cursor()  
cur.execute('''CREATE TABLE corpus_sents  
     (id SERIAL,
     title TEXT NOT NULL,
     n_sents INT NOT NULL);''')

print("Table created successfully")
con.commit()  
con.close()

Table created successfully


In [73]:
%%time
con = psycopg2.connect(
  database="russian-news", 
  user="postgres", 
  password="passcode4854", 
  host="127.0.0.1", 
  port="5432"
)
user_records = ", ".join(["%s"] * len(n_sents))

insert_query = (
    f"INSERT INTO corpus_sents (title, n_sents) VALUES {user_records}"
)

cursor = con.cursor()
cursor.execute(insert_query, n_sents)
con.commit()  
con.close()

Wall time: 89.8 ms


## 2. Манипуляция данными

Заменим все статьи Ленты, которые были написаны в 2001 году, на 0:

In [74]:
%%time
con = psycopg2.connect(
  database="russian-news", 
  user="postgres", 
  password="passcode4854", 
  host="127.0.0.1", 
  port="5432"
)
cur = con.cursor()
cur.execute("UPDATE corpus SET article = 0 WHERE date LIKE '2001%' AND newspaper = 'lenta'")
con.commit()  
con.close()

Wall time: 102 ms


Удалим все статьи Ленты за 2001 год:

In [75]:
%%time
con = psycopg2.connect(
  database="russian-news", 
  user="postgres", 
  password="passcode4854", 
  host="127.0.0.1", 
  port="5432"
)
cur = con.cursor()
cur.execute("DELETE FROM corpus WHERE date LIKE '2001%' AND newspaper = 'lenta'")
con.commit()  
con.close()

Wall time: 85.8 ms


Выберем все статьи Известий, количество слов в которых больше среднего количества слов по корпусу.
Для этого выполним многотабличный запрос: обратимся к таблицам corpus и corpus_words с применением RIGHT JOIN (потому что мы удалили некоторые статьи Ленты из таблицы corpus и не для каждой строки из corpus_words найдется соответствующая строка из corpus.
Чтобы столбцы таблиц не дублировались (в некоторых содержатся одинаковые значения), в явном виде пропишем, какие столбцы из каких таблиц мы хотим видеть:

In [77]:
%%time
con = psycopg2.connect(
  database="russian-news", 
  user="postgres", 
  password="passcode4854", 
  host="127.0.0.1", 
  port="5432"
)
cur = con.cursor()
cur.execute("SELECT corpus_words.id, newspaper, date, corpus_words.title, article, corpus_words.n_words FROM corpus RIGHT JOIN corpus_words ON corpus.title = corpus_words.title WHERE n_words > (SELECT AVG(n_words) FROM corpus_words) ORDER BY id")
data=cur.fetchall()
print(data)
con.close()

[(9, None, None, 'Российская экономика третьего тысячелетия: новые деньги, новые зарплаты и новые налоги', None, 497), (93, None, None, 'Большой праздник на NASDAQ: акции компаний хай-тек снова в цене', None, 369), (117, 'lenta', '2002-01-01T22:04:58+03:00', 'Экономика Израиля пережила самый плохой год за всю историю страны', 'В сфере экономики 2001 год был для Израиля худшим почти за всю историю страны. Об этом свидетельствуют показатели, опубликованные в понедельник израильским ЦСУ.Впервые за последние 48 лет прирост ВВП оказался отрицательным и составил -0,5% (в 2000 году тот же показатель вырос на 6,4%). Последний раз падение израильского ВВП было отмечено лишь в 1953 году (-1,4%). ВВП на душу населения снизился в уходящем году на 2,9% и составил 17.100 долларов (для сравнения, в России - порядка 1200 долларов). Объем израильского экспорта упал на 13,1%, а импорт сократился на 6,4%.Спад в экономике Израиля вызвал резкое снижение доходов местных коммерческих банков. В связи с

Теперь выберем все статьи РИА-новостей, кол-во слов и предложений в которой меньше среднего по корпусу.
Для этого привяжем с помощью RIGHT JOIN третью таблицу - corpus_sents:

In [78]:
%%time
con = psycopg2.connect(
  database="russian-news", 
  user="postgres", 
  password="passcode4854", 
  host="127.0.0.1", 
  port="5432"
)
cur = con.cursor()
cur.execute("SELECT corpus_words.id, newspaper, date, corpus_words.title, article, corpus_words.n_words, corpus_sents.n_sents FROM corpus RIGHT JOIN corpus_words ON corpus.title = corpus_words.title RIGHT JOIN corpus_sents ON corpus.title = corpus_sents.title WHERE n_words < (SELECT AVG(n_words) FROM corpus_words) AND n_sents < (SELECT AVG(n_sents) FROM corpus_sents) AND corpus.newspaper = 'ria' ORDER BY id")
data=cur.fetchall()
print(data)
con.close()

[(601, 'ria', '2001-10-16T17:55:00+03:00', 'На выборах губернатора Орловской области партия "Единство" поддерживает кандидатуру Егора Строева', 'На выборах губернатора Орловской области, намеченных на 28 октября, партия "Единство" поддерживает кандидатуру нынешнего главы областной администрации, спикера Совета Федерации Егора Строева. Об этом заявил журналистам во вторник исполняющий обязанности председателя политсовета "Единство" Франц Клинцевич. На заседании президиума политсовета, состоявшемся во вторник, рассматривались итоги выборов в регионах, а также вопросы подготовки к третьему съезду "Единства", который должен состояться 27 октября, и объединительному съезду "Единство" и "Отечество". Сразу же после окончания заседания президиума политсовета "Единство" началось заседание генерального совета Союза "Единство" и "Отечество". В заседании, которое проходит в закрытом режиме, принимают участие лидеры двух организаций Сергей Шойгу и Юрий...\n МОСКВА, 16 октября. /Корр. РИА "

Запишем полученные данные в новую таблицу - new_corpus:

In [79]:
con = psycopg2.connect(
  database="russian-news", 
  user="postgres", 
  password="passcode4854", 
  host="127.0.0.1", 
  port="5432"
)
cur = con.cursor()  
cur.execute('''CREATE TABLE new_corpus  
     (id INT NOT NULL,
     newspaper TEXT,
     date TEXT,
     title TEXT NOT NULL,
     article TEXT,
     n_words INT NOT NULL,
     n_sents INT NOT NULL);''')

print("Table created successfully")
con.commit()  
con.close()

Table created successfully


In [80]:
%%time
con = psycopg2.connect(
  database="russian-news", 
  user="postgres", 
  password="passcode4854", 
  host="127.0.0.1", 
  port="5432"
)
user_records = ", ".join(["%s"] * len(data))

insert_query = (
    f"INSERT INTO new_corpus (id, newspaper, date, title, article, n_words, n_sents) VALUES {user_records}"
)

cursor = con.cursor()
cursor.execute(insert_query, data)
con.commit()  
con.close()

Wall time: 114 ms


Столбец id неинформативен, можно его удалить:

In [81]:
%%time
con = psycopg2.connect(
  database="russian-news", 
  user="postgres", 
  password="passcode4854", 
  host="127.0.0.1", 
  port="5432"
)
cur = con.cursor()
cur.execute("ALTER TABLE new_corpus DROP COLUMN id")
con.commit()  
con.close()

Wall time: 88.4 ms


In [ ]:
Таблицу corpus тоже удалим:

In [82]:
%%time
con = psycopg2.connect(
  database="russian-news", 
  user="postgres", 
  password="passcode4854", 
  host="127.0.0.1", 
  port="5432"
)
cur = con.cursor()
cur.execute("DROP TABLE corpus")
con.commit()  
con.close()

Wall time: 103 ms


list